In [64]:
import polars as pl
import sys
import os

print("--- Lancement Data Prep (Version Notebook) ---")

# --- 1. DÉFINIR LES CHEMINS (Version Notebook) ---
# On part de /Notebooks/ et on remonte avec "../"
PATH_UL = "../Data/raw/StockUniteLegale_utf8.parquet"
PATH_ETAB = "../Data/raw/StockEtablissement_utf8.parquet"
PATH_ETAB_HISTO = "../Data/raw/StockEtablissementHistorique_utf8.parquet"
PATH_OUTPUT = "../Data/processed/sirene_infos_MONSTROUS_DEMO.parquet" # C'est notre nouveau "Master File"

print("--- Lancement Script 01: Création du MASTER FILE 'Démo Monstrueuse' ---")

# --- 2. VÉRIFICATION DES FICHIERS ---
for path in [PATH_UL, PATH_ETAB, PATH_ETAB_HISTO]:
    if not os.path.exists(path):
        print(f"ERREUR FATALE: Fichier brut manquant : {path}", file=sys.stderr)
        print("Vérifie que tes 3 fichiers SIRENE sont bien dans '../Data/raw/'")
        raise FileNotFoundError(f"Fichier non trouvé : {path}")
print("Fichiers bruts trouvés. Lancement de la pipeline...")

# ===================================================================
# ÉTAPE 1: La Base (FEATURES X) - Fichier 'StockUniteLegale'
# (On prend 8 features au lieu de 5)
# ===================================================================
print("Étape 1: Lecture des features de 'StockUniteLegale'...")
df_base_features = pl.scan_parquet(PATH_UL).select(
    "siren",
    "dateCreationUniteLegale",
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",                 # <-- AJOUTÉ
    "economieSocialeSolidaireUniteLegale", # <-- AJOUTÉ
    "societeMissionUniteLegale"            # <-- AJOUTÉ
)

# ===================================================================
# ÉTAPE 2: Trouver le SIRET du Siège (HQ) - Fichier 'StockEtablissement'
# (On prend 3 features au lieu d'une)
# ===================================================================
print("Étape 2: Lecture de 'StockEtablissement' pour trouver les sièges...")
df_sieges = pl.scan_parquet(PATH_ETAB).filter(
    pl.col("etablissementSiege") == True
).select(
    "siren", 
    "siret",
    pl.col("codePostalEtablissement").str.slice(0, 2).alias("departement"),
    pl.col("trancheEffectifsEtablissement").alias("trancheEffectifsSiege"), # <-- NOUVELLE FEATURE
    pl.col("caractereEmployeurEtablissement").alias("caractereEmployeurSiege") # <-- NOUVELLE FEATURE
)

# ===================================================================
# ÉTAPE 3: Trouver la Date de Fermeture (La Cible Y) - Fichier 'StockEtablissementHistorique'
# (Rien ne change ici)
# ===================================================================
print("Étape 3: Lecture de 'StockEtablissementHistorique' pour trouver les 'morts'...")
df_fermetures = pl.scan_parquet(PATH_ETAB_HISTO).filter(
    pl.col("etatAdministratifEtablissement") == 'F'
).select(
    "siret",
    pl.col("dateFin").alias("dateFermeture")
).group_by("siret").agg(
    pl.col("dateFermeture").max() 
)

# ===================================================================
# ÉTAPE 4: Le "Grand Mariage" SIRENE
# (Rien ne change ici)
# ===================================================================
print("Étape 4: Jointure finale des 3 tables...")
# On utilise .collect() ici pour forcer la 1ère jointure
df_master = df_base_features.collect().join(
    df_sieges.collect(), on="siren", how="left"
)
# On joint le 2e
df_master = df_master.join(
    df_fermetures.collect(), on="siret", how="left"
)

# ===================================================================
# ÉTAPE 5: Sauvegarde
# (On a maintenant 14 colonnes "propres" !)
# ===================================================================
print(f"Sauvegarde du Master File 'Démo Monstrueuse' dans {PATH_OUTPUT}...")
df_final = df_master.select(
    # Les Clés (pour la Target)
    "siren",
    "dateCreationUniteLegale",
    "dateFermeture",
    # Les 12 Features "Monstrueuses"
    "categorieJuridiqueUniteLegale",
    "trancheEffectifsUniteLegale",
    "activitePrincipaleUniteLegale",
    "categorieEntreprise",
    "economieSocialeSolidaireUniteLegale",
    # "societeMissionUniteLegale",
    # "sexeUniteLegale", # On l'enlève, il est souvent 'null'
    pl.col("dateCreationUniteLegale").dt.year().alias("anneeCreation"),
    pl.col("dateCreationUniteLegale").dt.month().alias("moisCreation"),
    "departement",
    "trancheEffectifsSiege",
    "caractereEmployeurSiege"
)

# On s'assure que le dossier 'processed' existe
os.makedirs(os.path.dirname(PATH_OUTPUT), exist_ok=True)

# On sauvegarde
df_final.write_parquet(PATH_OUTPUT)
print(f"--- Script 01 (Démo Monstrueuse) Terminé avec Succès ---")
print(f"Fichier créé : {PATH_OUTPUT}")
print(f"Shape finale : {df_final.shape}")
print(df_final.head())

--- Lancement Data Prep (Version Notebook) ---
--- Lancement Script 01: Création du MASTER FILE 'Démo Monstrueuse' ---
Fichiers bruts trouvés. Lancement de la pipeline...
Étape 1: Lecture des features de 'StockUniteLegale'...
Étape 2: Lecture de 'StockEtablissement' pour trouver les sièges...
Étape 3: Lecture de 'StockEtablissementHistorique' pour trouver les 'morts'...
Étape 4: Jointure finale des 3 tables...
Sauvegarde du Master File 'Démo Monstrueuse' dans ../Data/processed/sirene_infos_MONSTROUS_DEMO.parquet...
--- Script 01 (Démo Monstrueuse) Terminé avec Succès ---
Fichier créé : ../Data/processed/sirene_infos_MONSTROUS_DEMO.parquet
Shape finale : (28882409, 13)
shape: (5, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ dateCreat ┆ dateFerme ┆ categorie ┆ … ┆ moisCreat ┆ departeme ┆ trancheEf ┆ caracter │
│ ---       ┆ ionUniteL ┆ ture      ┆ Juridique ┆   ┆ ion       ┆ nt        ┆ fectifsSi ┆ eEmploye │
│ str

In [65]:
df_final.describe()

statistic,siren,dateCreationUniteLegale,dateFermeture,categorieJuridiqueUniteLegale,trancheEffectifsUniteLegale,activitePrincipaleUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale,anneeCreation,moisCreation,departement,trancheEffectifsSiege,caractereEmployeurSiege
str,str,str,str,f64,str,str,str,str,f64,f64,str,str,str
"""count""","""28882409""","""27710121""","""1198759""",2.8882409e7,"""28882409""","""28860884""","""10546073""","""6167001""",2.7710121e7,2.7710121e7,"""28560617""","""28860588""","""28707092"""
"""null_count""","""0""","""1172288""","""27683650""",0.0,"""0""","""21525""","""18336336""","""22715408""",1.172288e6,1.172288e6,"""321792""","""21821""","""175317"""
"""mean""",null,"""2005-10-22 13:11:56.004134""","""2010-09-25 00:21:57.306314""",3246.573881,null,null,null,null,2005.370016,5.966385,null,null,null
"""std""",null,null,null,2736.41353,null,null,null,null,18.148138,3.714184,null,null,null
"""min""","""000325175""","""0001-01-16""","""1900-12-31""",1000.0,"""00""","""00.00""","""ETI""","""N""",1.0,1.0,""" D""","""00""","""N"""
"""25%""",null,"""1995-01-19""","""2003-12-24""",1000.0,null,null,null,null,1995.0,2.0,null,null,null
"""50%""",null,"""2010-05-01""","""2010-10-13""",1000.0,null,null,null,null,2010.0,6.0,null,null,null
"""75%""",null,"""2019-11-02""","""2019-12-02""",5599.0,null,null,null,null,2019.0,9.0,null,null,null
"""max""","""999992357""","""3023-01-06""","""5015-04-04""",9970.0,"""NN""","""99.0Z""","""PME""","""O""",3023.0,12.0,"""sw""","""NN""","""O"""


In [66]:
# je veux clean toutes les nulls dans df_final des colonnes departement, dateCreationUniteLegale, activitéPrincipaleUniteLegale, trancheEffectifsUniteLegale

df_clean = df_final.drop_nulls(subset=[
    "departement", 
    "dateCreationUniteLegale", 
    "activitePrincipaleUniteLegale", 
    "trancheEffectifsUniteLegale",
    "categorieEntreprise",
    "caractereEmployeurSiege"
])

In [67]:
# je veux que tu m'enleve toutes les lignes ou la date de fermeture est avant la date de création masi aussi ou la date de fermeture est le même jour que la date de création mais aussi si la date de fermeture est plus tard que la date du jour 

from datetime import datetime
today = datetime.today().date()

df_clean = df_clean.filter(
    ( (pl.col("dateFermeture").is_null()) | 
      (pl.col("dateFermeture") > pl.col("dateCreationUniteLegale")) & 
      (pl.col("dateFermeture") < pl.lit(today)) )
)

In [68]:
# Je veux enlever les entreprises crées avant 1970 et après la date d'ajourd'hui

df_clean = df_clean.filter(
    (pl.col("dateCreationUniteLegale").dt.year() >= 1970) & 
    (pl.col("dateCreationUniteLegale") <= pl.lit(today))
)

In [69]:
df_clean.describe()

statistic,siren,dateCreationUniteLegale,dateFermeture,categorieJuridiqueUniteLegale,trancheEffectifsUniteLegale,activitePrincipaleUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale,anneeCreation,moisCreation,departement,trancheEffectifsSiege,caractereEmployeurSiege
str,str,str,str,f64,str,str,str,str,f64,f64,str,str,str
"""count""","""10223433""","""10223433""","""432318""",1.0223433e7,"""10223433""","""10223433""","""10223433""","""2902459""",1.0223433e7,1.0223433e7,"""10223433""","""10223433""","""10223433"""
"""null_count""","""0""","""0""","""9791115""",0.0,"""0""","""0""","""0""","""7320974""",0.0,0.0,"""0""","""0""","""0"""
"""mean""",null,"""2012-06-08 22:07:57.438449""","""2018-11-30 19:07:51.502921""",3056.154825,null,null,null,null,2011.980528,6.185161,null,null,null
"""std""",null,null,null,2549.956592,null,null,null,null,11.055441,3.593599,null,null,null
"""min""","""000325175""","""1970-01-01""","""1971-04-19""",1000.0,"""00""","""00.00Z""","""ETI""","""N""",1970.0,1.0,""" D""","""00""","""N"""
"""25%""",null,"""2007-05-10""","""2017-01-02""",1000.0,null,null,null,null,2007.0,3.0,null,null,null
"""50%""",null,"""2016-02-01""","""2020-12-31""",1000.0,null,null,null,null,2016.0,6.0,null,null,null
"""75%""",null,"""2021-02-11""","""2022-12-31""",5499.0,null,null,null,null,2021.0,9.0,null,null,null
"""max""","""999992357""","""2025-11-01""","""2025-11-17""",9970.0,"""NN""","""99.00Z""","""PME""","""O""",2025.0,12.0,"""sw""","""NN""","""O"""


In [70]:
df_clean.head()

siren,dateCreationUniteLegale,dateFermeture,categorieJuridiqueUniteLegale,trancheEffectifsUniteLegale,activitePrincipaleUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale,anneeCreation,moisCreation,departement,trancheEffectifsSiege,caractereEmployeurSiege
str,date,date,i64,str,str,str,str,i32,i8,str,str,str
"""000325175""",2000-09-26,null,1000,"""NN""","""32.12Z""","""PME""",null,2000,9,"""13""","""NN""","""N"""
"""005540273""",1972-01-01,null,1000,"""NN""","""68.20B""","""PME""",null,1972,1,"""04""","""NN""","""N"""
"""005541552""",1974-01-01,null,5710,"""02""","""56.10C""","""PME""","""N""",1974,1,"""04""","""01""","""O"""
"""005641154""",1981-02-01,null,1000,"""NN""","""68.31Z""","""PME""",null,1981,2,"""04""","""NN""","""N"""
"""005742580""",1993-03-01,null,1000,"""NN""","""68.20B""","""PME""",null,1993,3,"""04""","""NN""","""N"""


In [71]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. CHARGER LE "MASTER FILE" DÉMO MONSTRUEUX ---
PATH_ML_DEMO = "../Data/processed/sirene_infos_MONSTROUS_DEMO.parquet"
try:
    df_master = pl.read_parquet(PATH_ML_DEMO)
    print(f"Dataset 'Démo Monstrueuse' chargé. Shape: {df_master.shape}")
except FileNotFoundError:
    print(f"ERREUR: Fichier non trouvé : {PATH_ML_DEMO}")
    print("Assure-toi que ton script de Data Prep a bien tourné.")

# --- 2. FILTRER LA COHORTE ---
# On va étudier les entreprises créées en 2018
print("Filtrage de la Cohorte 2018...")
df_ml = df_master.filter(
    pl.col("dateCreationUniteLegale").dt.year() == 2018
)
print(f"Cohorte 2018 isolée. Shape: {df_ml.shape}")

# --- 3. CRÉATION DE LA TARGET (Y) ---
print("Création de la Cible (is_failed_in_3y)...")

df_ml = df_ml.with_columns(
    # Date limite = 3 ans après la création
    (pl.col("dateCreationUniteLegale").dt.offset_by("3y")).alias("date_limite_3_ans")
).with_columns(
    # 1 = Faillite si Fermé AVANT la date limite
    pl.when(
        (pl.col("dateFermeture").is_not_null()) & # dateFermeture n'est pas null
        (pl.col("dateFermeture") < pl.col("date_limite_3_ans"))
    ).then(1)
    .otherwise(0)
    .alias("is_failed_in_3y")
).fill_null("INCONNU") # Nettoyage final des features catégorielles

# --- 4. VÉRIFICATION (EDA Rapide) ---
print("--- DATASET ML PRÊT ---")
print(f"Shape finale prête pour le ML : {df_ml.shape}")

# On vérifie la répartition de notre nouvelle Cible
print("\nRépartition de la Cible (Y) :")
print(df_ml.get_column("is_failed_in_3y").value_counts())

print("\nAperçu du DataFrame final :")
print(df_ml.head())

Dataset 'Démo Monstrueuse' chargé. Shape: (28882409, 13)
Filtrage de la Cohorte 2018...
Cohorte 2018 isolée. Shape: (845851, 13)
Création de la Cible (is_failed_in_3y)...
--- DATASET ML PRÊT ---
Shape finale prête pour le ML : (845851, 15)

Répartition de la Cible (Y) :
shape: (2, 2)
┌─────────────────┬────────┐
│ is_failed_in_3y ┆ count  │
│ ---             ┆ ---    │
│ i32             ┆ u32    │
╞═════════════════╪════════╡
│ 1               ┆ 11869  │
│ 0               ┆ 833982 │
└─────────────────┴────────┘

Aperçu du DataFrame final :
shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ dateCreat ┆ dateFerme ┆ categorie ┆ … ┆ trancheEf ┆ caractere ┆ date_limi ┆ is_faile │
│ ---       ┆ ionUniteL ┆ ture      ┆ Juridique ┆   ┆ fectifsSi ┆ Employeur ┆ te_3_ans  ┆ d_in_3y  │
│ str       ┆ egale     ┆ ---       ┆ UniteLega ┆   ┆ ege       ┆ Siege     ┆ ---       ┆ ---      │
│           ┆ ---       ┆ date     

In [72]:
df_ml.describe()

statistic,siren,dateCreationUniteLegale,dateFermeture,categorieJuridiqueUniteLegale,trancheEffectifsUniteLegale,activitePrincipaleUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale,anneeCreation,moisCreation,departement,trancheEffectifsSiege,caractereEmployeurSiege,date_limite_3_ans,is_failed_in_3y
str,str,str,str,f64,str,str,str,str,f64,f64,str,str,str,str,f64
"""count""","""845851""","""845851""","""27598""",845851.0,"""845851""","""845851""","""845851""","""845851""",845851.0,845851.0,"""845851""","""845851""","""845851""","""845851""",845851.0
"""null_count""","""0""","""0""","""818253""",0.0,"""0""","""0""","""0""","""0""",0.0,0.0,"""0""","""0""","""0""","""0""",0.0
"""mean""",null,"""2018-06-19 15:03:25.836252""","""2021-09-18 01:42:16.096818""",3199.927141,null,null,null,null,2018.0,6.25518,null,null,null,"""2021-06-19 15:03:25.836252""",0.014032
"""std""",null,null,null,2668.588219,null,null,null,null,0.0,3.526369,null,null,null,null,0.117623
"""min""","""130023385""","""2018-01-01""","""2017-12-31""",1000.0,"""00""","""00.00Z""","""ETI""","""INCONNU""",2018.0,1.0,"""01""","""00""","""INCONNU""","""2021-01-01""",0.0
"""25%""",null,"""2018-03-15""","""2020-05-30""",1000.0,null,null,null,null,2018.0,3.0,null,null,null,"""2021-03-15""",0.0
"""50%""",null,"""2018-06-18""","""2021-11-03""",1000.0,null,null,null,null,2018.0,6.0,null,null,null,"""2021-06-18""",0.0
"""75%""",null,"""2018-09-28""","""2023-02-05""",5710.0,null,null,null,null,2018.0,9.0,null,null,null,"""2021-09-28""",0.0
"""max""","""993394725""","""2018-12-31""","""2025-10-31""",9970.0,"""NN""","""99.00Z""","""PME""","""O""",2018.0,12.0,"""[N""","""NN""","""O""","""2021-12-31""",1.0


In [73]:
# sauvergarde ça dans un parquet

PATH_OUTPUT_CLEAN = "../Data/processed/sirene_infos_CLEAN.parquet"

df_ml.write_parquet(PATH_OUTPUT_CLEAN)


In [74]:
df_ml.schema

Schema([('siren', String),
        ('dateCreationUniteLegale', Date),
        ('dateFermeture', Date),
        ('categorieJuridiqueUniteLegale', Int64),
        ('trancheEffectifsUniteLegale', String),
        ('activitePrincipaleUniteLegale', String),
        ('categorieEntreprise', String),
        ('economieSocialeSolidaireUniteLegale', String),
        ('anneeCreation', Int32),
        ('moisCreation', Int8),
        ('departement', String),
        ('trancheEffectifsSiege', String),
        ('caractereEmployeurSiege', String),
        ('date_limite_3_ans', Date),
        ('is_failed_in_3y', Int32)])